# Original code by Philipp Muellauer
<p> https://www.udemy.com/course/python-data-science-machine-learning-bootcamp/?couponCode=LEADERSALE24A </p>

# Imports

In [1]:
import os
import numpy as np
import tensorflow.compat.v1 as tf

from time import strftime
from PIL import Image

2024-05-23 19:16:00.754866: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-23 19:16:01.357367: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Constants

In [2]:
NUM_OF_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_WIDTH*IMAGE_HEIGHT*CHANNELS
BATCH_SIZE = 1000

# Load data

In [3]:
x_train_full = np.loadtxt('data/digit_xtrain.csv', delimiter=',', dtype=int)

In [4]:
x_test = np.loadtxt('data/digit_xtest.csv', delimiter=',', dtype=int)

In [5]:
y_train_full = np.loadtxt('data/digit_ytrain.csv', delimiter=',', dtype=int)

In [6]:
y_test = np.loadtxt('data/digit_ytest.csv', delimiter=',', dtype=int)

# Preprocessing

In [7]:
# Normalization (rescale)
x_train_full = x_train_full / 255.0
x_test = x_test / 255.0
print(x_test[0])

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         

In [8]:
#One-hot encoding
y_train_full = np.eye(NUM_OF_CLASSES)[y_train_full]
y_test = np.eye(NUM_OF_CLASSES)[y_test]


In [9]:
# Create subsets for traning and validation
x_valid = x_train_full[:VALIDATION_SIZE]
y_valid = y_train_full[:VALIDATION_SIZE]

In [10]:
x_train = x_train_full[VALIDATION_SIZE:]
y_train = y_train_full[VALIDATION_SIZE:]

# Create the Graph

In [11]:
# To be able to create the graph
tf.disable_eager_execution()

In [12]:
X = tf.placeholder(tf.float32, shape=[None, TOTAL_INPUTS], name='X')
Y = tf.placeholder(tf.float32, shape=[None, NUM_OF_CLASSES], name='labels')

# Architecture

In [13]:
# Hyperparameters
num_of_epochs = 50
learning_rate = 1e-3

nodes_hidden1 = 512
nodes_hidden2 = 64

In [14]:
# Init parameters
def setup_layer(input, weight_dim, bias_dim, name):
    
    with tf.name_scope(name):
        init_weight = tf.truncated_normal(shape=weight_dim, stddev=0.1)
        w = tf.Variable(initial_value=init_weight, name='W')

        init_bias = tf.constant(value=0.0, shape=bias_dim)
        b = tf.Variable(initial_value=init_bias, name='B')
        
        input_layer = tf.matmul(input, w) + b
        
        if name == 'out':
            output_layer = tf.nn.softmax(input_layer)
        else:
            output_layer = tf.nn.relu(input_layer)
        
        tf.summary.histogram('weights', w)
        tf.summary.histogram('biases', b)
        
        return output_layer

In [15]:
# Create layers
layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, nodes_hidden1], 
                      bias_dim=[nodes_hidden1], name='layer_1')

layer_dropout = tf.nn.dropout(layer_1, rate=0.8, name='dropout_layer')

layer_2 = setup_layer(layer_dropout, weight_dim=[nodes_hidden1, nodes_hidden2], 
                      bias_dim=[nodes_hidden2], name='layer_2')

output = setup_layer(layer_2, weight_dim=[nodes_hidden2, NUM_OF_CLASSES], 
                      bias_dim=[NUM_OF_CLASSES], name='out')

model_name = f'N1-{nodes_hidden1}_Drop_N2-{nodes_hidden2}_LR-{learning_rate}_E-{num_of_epochs}'

In [16]:
# Folder for Tensorboard

folder_name = f'{model_name}_Time-{strftime("%H:%M")}'
directory = os.path.join("logs/", folder_name)

try:
    os.makedirs(directory)
except OSError as exception:
    print(exception)
else:
    print('Successfully created directories!')

Successfully created directories!


# Loss Function

In [17]:
with tf.name_scope('loss_calc'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output))

# Optimizer

In [18]:
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_step = optimizer.minimize(loss)

# Accuracy Metrics

In [19]:
# Elementwise comparison of Y and the predictions
with tf.name_scope('accuracy_calc'):
    model_prediction = tf.argmax(output, axis=1, name="prediction")
    correct_pred = tf.equal(model_prediction, tf.argmax(Y, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [20]:
with tf.name_scope('performance'):
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar('cost', loss)

In [21]:
with tf.name_scope('show_image'):
    x_image = tf.reshape(X, [-1, 28, 28, 1])
    tf.summary.image('image_input', x_image, max_outputs=4)

# Run Session

In [22]:
session = tf.Session()

In [23]:
merged_summary = tf.summary.merge_all()

train_writer = tf.summary.FileWriter(directory + '/train')
train_writer.add_graph(session.graph)

validation_writer = tf.summary.FileWriter(directory + 'validation')

In [24]:
# Add the variables
init = tf.global_variables_initializer()
session.run(init)

2024-05-23 19:16:03.768616: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


In [25]:
# Creating batches
num_of_samples = y_train.shape[0]
num_of_iterations = int(num_of_samples/BATCH_SIZE)

index_in_epoch = 0

In [26]:
def next_batch(batch_size, data, labels):
    
    global num_of_samples
    global index_in_epoch
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > num_of_samples:
        start = 0
        index_in_epoch = batch_size
    
    end = index_in_epoch
    
    return data[start:end], labels[start:end]

In [27]:
# Loops
for epoch in range(num_of_epochs):
    
    # Training
    for i in range(num_of_iterations):
        
        batch_x, batch_y = next_batch(batch_size=BATCH_SIZE, data=x_train, labels=y_train)
        
        feed_dictionary = {X:batch_x, Y:batch_y}
        
        session.run(train_step, feed_dict=feed_dictionary)
        
    
    s, batch_accuracy = session.run(fetches=[merged_summary, accuracy], feed_dict=feed_dictionary)
        
    train_writer.add_summary(s, epoch)
    
    print(f'Epoch {epoch} \t| Training Accuracy = {batch_accuracy}')
    
    # Validation
    summary = session.run(fetches=merged_summary, feed_dict={X:x_valid, Y:y_valid})
    validation_writer.add_summary(summary, epoch)

print('Done training!')

Epoch 0 	| Training Accuracy = 0.7960000038146973
Epoch 1 	| Training Accuracy = 0.902999997138977
Epoch 2 	| Training Accuracy = 0.9279999732971191
Epoch 3 	| Training Accuracy = 0.9399999976158142
Epoch 4 	| Training Accuracy = 0.9390000104904175
Epoch 5 	| Training Accuracy = 0.9459999799728394
Epoch 6 	| Training Accuracy = 0.9470000267028809
Epoch 7 	| Training Accuracy = 0.953000009059906
Epoch 8 	| Training Accuracy = 0.9559999704360962
Epoch 9 	| Training Accuracy = 0.953000009059906
Epoch 10 	| Training Accuracy = 0.9430000185966492
Epoch 11 	| Training Accuracy = 0.9570000171661377
Epoch 12 	| Training Accuracy = 0.9679999947547913
Epoch 13 	| Training Accuracy = 0.9629999995231628
Epoch 14 	| Training Accuracy = 0.9559999704360962
Epoch 15 	| Training Accuracy = 0.9629999995231628
Epoch 16 	| Training Accuracy = 0.9649999737739563
Epoch 17 	| Training Accuracy = 0.9660000205039978
Epoch 18 	| Training Accuracy = 0.9750000238418579
Epoch 19 	| Training Accuracy = 0.9599999785

# Saving the model

In [28]:
outputs = {"accuracy_calc/prediction": model_prediction}
inputs = {"X": X}

tf.saved_model.simple_save(session, "saved_model", inputs, outputs)

# Make sure the session is running!!!


Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.
Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: saved_model/saved_model.pb


# Make a Prediction

In [29]:
img = Image.open('data/test_img.png')
img

In [30]:
# Luminance mode --> Black & white
greyscale_pic = img.convert('L')

In [31]:
# Swap blacks and whites
img_array = np.invert(greyscale_pic)

In [32]:
# Make it 1D
test_img = img_array.ravel()

In [33]:
# Make a prediction
prediction = session.run(fetches=tf.argmax(output, axis=1), feed_dict={X:[test_img]})
print(f'Prediction for test image is {prediction}')

Prediction for test image is [2]


# Testing

In [34]:
test_accuracy = session.run(fetches=accuracy, feed_dict={X:x_test, Y:y_test})
print(f'Accuracy on test set is {test_accuracy:0.2%}')

Accuracy on test set is 95.72%


# Reset Tensorboard

In [35]:
train_writer.close()
validation_writer.close()
session.close()
tf.reset_default_graph()